In [1]:
# Install packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5

In [2]:
!pip install -U langchain-community langchain-core

In [5]:
# Imports
import os
from google.colab import userdata
import langchain, langchain_core
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [6]:
# Set API Key
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY_DEFAULT")

In [7]:
# 1. Create Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7
)

In [8]:
# 2. Define a Trimmer (The new "Window Memory")
# This replaces k=2 by keeping the last 5 messages (adjust as needed)
trimmer = trim_messages(
    max_tokens=5, # Simplified here for "last few messages" logic
    strategy="last",
    token_counter=len, # Counts messages rather than raw tokens for simplicity
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [9]:
# 3. Define the Prompt Template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

In [10]:
# 4. Create the Chain using LCEL
# We use the trimmer *inside* the history management or as part of the chain
chain = prompt | llm


In [11]:
# 5. Manage History (The "Memory" container)
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [12]:
config = {"configurable": {"session_id": "user_1"}}

# Step 1: Name input
name = input("Enter your name: ")
wrapped_chain.invoke({"input": f"My name is {name}. Please remember it and greet me."}, config)

# Step 2: Topic 1
topic1 = wrapped_chain.invoke({"input": "Tell me something interesting about Mars."}, config)
print("\nTopic 1:\n", topic1.content)

# Step 3: Topic 2
topic2 = wrapped_chain.invoke({"input": "Now explain how plants make food."}, config)
print("\nTopic 2:\n", topic2.content)

# Step 4: Topic 3
topic3 = wrapped_chain.invoke({"input": "Now explain Where do we apply calculus in Data Science?."}, config)
print("\nTopic 3:\n", topic3.content)

# Step 5: The Test (Will it remember the name?)
# Because of the sliding window/trimming, the earliest messages will be dropped.
followup = wrapped_chain.invoke({"input": "Do you remember my name?"}, config)
print("\nFollow-up Response:\n", followup.content)

Enter your name: John

Topic 1:
 Did you know that Mars is home to the **largest volcano in the entire solar system**?

It's called **Olympus Mons**, and it's a massive shield volcano. To give you an idea of its scale:

*   It's about **25 kilometers (16 miles) high**, which is nearly three times the height of Mount Everest!
*   It has a diameter of about **600 kilometers (370 miles)**, making it wide enough to cover an area roughly the size of Arizona.

It's an incredible geological feature!

Topic 2:
 That's a great question! Plants are incredible because they can make their own food through a process called **photosynthesis**.

Think of a plant as a tiny, incredibly efficient food factory. Here's how it works:

1.  **The Ingredients (What a plant needs):**
    *   **Sunlight:** This is the energy source, just like electricity for a factory. Plants absorb sunlight, usually through their leaves.
    *   **Water:** Absorbed from the soil through its roots and transported up to the leav